# Импорт библиотек

In [30]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

from sqlalchemy import create_engine, inspect
from ydata_profiling import ProfileReport

In [31]:
# pd.options.plotting.backend = "matplotlib"

In [42]:
import yaml
from os import path, makedirs

# Подключение к базе данных

In [33]:
# чтобы выложить проект на гитхабе и не палить параметры подключения,
# загрузим их из yml файла (файл добавлен в .gitignore)
with open('db_config.yml', 'r') as file:
    db_config = yaml.safe_load(file)

# строка для подключения к базе
connection_string = 'postgresql://{}:{}@{}:{}/{}'.format(
    db_config['user'],
    db_config['pwd'],
    db_config['host'],
    db_config['port'],
    db_config['db'],
)

In [34]:
engine = create_engine(connection_string) 

In [35]:
inspector = inspect(engine)
table_names = inspector.get_table_names()

for table_name in table_names:
    columns = inspector.get_columns(table_name)
    print(f"Table: {table_name}")
    for column in columns:
        print(f"Column: {column['name']} - Type: {column['type']}")
    print("------------------------")

Table: collisions
Column: case_id - Type: TEXT
Column: county_city_location - Type: TEXT
Column: county_location - Type: TEXT
Column: distance - Type: REAL
Column: direction - Type: TEXT
Column: intersection - Type: INTEGER
Column: weather_1 - Type: TEXT
Column: location_type - Type: TEXT
Column: collision_damage - Type: TEXT
Column: party_count - Type: INTEGER
Column: primary_collision_factor - Type: TEXT
Column: pcf_violation_category - Type: TEXT
Column: type_of_collision - Type: TEXT
Column: motor_vehicle_involved_with - Type: TEXT
Column: road_surface - Type: TEXT
Column: road_condition_1 - Type: TEXT
Column: lighting - Type: TEXT
Column: control_device - Type: TEXT
Column: collision_date - Type: DATE
Column: collision_time - Type: TIME
------------------------
Table: case_ids
Column: case_id - Type: TEXT
Column: db_year - Type: TEXT
------------------------
Table: parties
Column: id - Type: INTEGER
Column: case_id - Type: TEXT
Column: party_number - Type: INTEGER
Column: party_ty

In [43]:
def download_tables(engine, table_names, datasets_folder='./datasets'):
    for table_name in table_names:

        if not path.exists(datasets_folder):
            makedirs(datasets_folder)
            
        dataset_path = datasets_folder + '/' + engine.url.database + '_' + table_name + '.csv'
        print("Start downloading tables:", table_names)

        if not path.exists(dataset_path):
            print("Downloading table:", dataset_path)
            query = '''SELECT * FROM {}'''.format(table_name)
            df = pd.read_sql_query(query, con=engine)
            df.to_csv(dataset_path)
            print("Download complete:", table_name)
            del df
        else:
            print(f"Table {table_name} is already downloaded:", dataset_path)
    
    print("Download comlete!")


In [44]:
download_tables(engine, table_names)

Start downloading tables: ['collisions', 'case_ids', 'parties', 'vehicles']
Download complete: collisions
Start downloading tables: ['collisions', 'case_ids', 'parties', 'vehicles']
Download complete: case_ids
Start downloading tables: ['collisions', 'case_ids', 'parties', 'vehicles']
Download complete: parties
Start downloading tables: ['collisions', 'case_ids', 'parties', 'vehicles']
Download complete: vehicles
Download comlete!


In [38]:
# процедура создание отчётов
def make_reports(engine, table_names, reports_folder='./reports'):
    
    for table_name in table_names:
        report_path = reports_folder + '/' + engine.url.database + '_' + table_name + '.html'
        print("Start reporting for tables:", table_names)
        
        if not path.exists(report_path):
            print('Forming report:', report_path)
            query = '''SELECT * FROM {}'''.format(table_name)
            df = pd.read_sql_query(query, con=engine)
            report = ProfileReport(df, minimal=True, missing_diagrams={'bar':False})
            report.to_file(report_path)
        else:
            print(f"Report for {table_name} already exists: {report_path}")
        
    print('Reporting completed!')
             